In [1]:
#in my Windows the optimizers are located in
#/c/Users/zb0857/AppData/Local/Continuum/anaconda2/envs/Tensorflow/lib/site-packages/tensorflow/python/keras
#C:\Users\zb0857\AppData\Local\Continuum\anaconda3\envs\ML\Lib\site-packages\tensorflow\python\keras
#/c/Users/zb0857/AppData/Local/Continuum/anaconda2/envs/Tensorflow/lib/site-packages/sklearn/neural_network
#C:\Users\zb0857\AppData\Local\Continuum\anaconda3\envs\ML\Lib\site-packages\sklearn\neural_network
#rememmber to start the notebook using the environment tensorflow


from numpy.random import seed
import numpy as np
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

import tensorflow as tf
# from tensorflow.python.keras.optimizers import Fire
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import optimizers

from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import log_loss

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable


import matplotlib.pyplot as plt
%matplotlib inline

## Preparing the input data


In [2]:
#Preparing the input data

#loading the MNIST dataset
mnist = tf.keras.datasets.mnist

#Separating into train and test (60000 train, 10000 test)
(x_train, y_train0),(x_test, y_test0) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

#flattening the images (from 28x28 to 784)
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

#we do not need to convert the outputs to one hot vectors.
#Also, if we convert the output to one hot vectors, the _label_binarizer.y_type_ 
#    becomes multilabel and the out_activation_ becomes logistic
#    Since keras and pytorch use softmax, we need to keep it like this
# #converting the otputs (labels) into one hot vectors
# lb = preprocessing.LabelBinarizer()
# lb.fit(np.array([0,1,2,3,4,5,6,7,8,9]))
# y_train = lb.transform(y_train0)
# y_test = lb.transform(y_test0)

## Defining the input parameters


In [3]:
#Defining the input parameters
BatchSize         = 5001
NeuronsLayer1     = 100
NeuronsLayer2     = 100
Epochs            = 100

## Model in SKLearn

In [4]:
# Defining the net
# Construct our model, # Construct our loss function and an Optimizer. 

#MLP regressor uses the square loss
#MLP classifier uses the log loss function
MLP_clf =  MLPClassifier( hidden_layer_sizes         = (NeuronsLayer1,NeuronsLayer2),
                          activation                 = 'relu',#,'identity', 'tanh', 'relu'],
                          solver                     = 'adam',
                          alpha                      = 0.0         ,
                          batch_size                 = BatchSize,
                          max_iter                   = Epochs              ,
                          random_state               = 1234,
                          tol                        = -10.0 ,
                          verbose                    = True,
                        )

In [5]:
#Show aprameters of the network
MLP_clf.get_params()

{'activation': 'relu',
 'alpha': 0.0,
 'batch_size': 5001,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100, 100),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_iter': 100,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 1234,
 'shuffle': True,
 'solver': 'adam',
 'tol': -10.0,
 'validation_fraction': 0.1,
 'verbose': True,
 'warm_start': False}

In [6]:
#train the network

MLP_clf.fit(x_train, y_train0)


Iteration 1, loss = 1.81390930
Iteration 2, loss = 0.89289847
Iteration 3, loss = 0.49292538
Iteration 4, loss = 0.36625217
Iteration 5, loss = 0.30977199
Iteration 6, loss = 0.27606573
Iteration 7, loss = 0.25161308
Iteration 8, loss = 0.23266756
Iteration 9, loss = 0.21603850
Iteration 10, loss = 0.20175424
Iteration 11, loss = 0.18910124
Iteration 12, loss = 0.17756110
Iteration 13, loss = 0.16786668
Iteration 14, loss = 0.15873063
Iteration 15, loss = 0.15019650
Iteration 16, loss = 0.14267300
Iteration 17, loss = 0.13554877
Iteration 18, loss = 0.12945584
Iteration 19, loss = 0.12341745
Iteration 20, loss = 0.11825523
Iteration 21, loss = 0.11312220
Iteration 22, loss = 0.10818262
Iteration 23, loss = 0.10334975
Iteration 24, loss = 0.09879028
Iteration 25, loss = 0.09429546
Iteration 26, loss = 0.09112211
Iteration 27, loss = 0.08690867
Iteration 28, loss = 0.08280716
Iteration 29, loss = 0.08051790
Iteration 30, loss = 0.07726292
Iteration 31, loss = 0.07414604
Iteration 32, los

C:\Users\zb0857\AppData\Local\Continuum\anaconda3\envs\ML\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0, batch_size=5001, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 100), learning_rate='constant',
              learning_rate_init=0.001, max_iter=100, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1234, shuffle=True, solver='adam', tol=-10.0,
              validation_fraction=0.1, verbose=True, warm_start=False)

In [7]:
#Predicting

y_train_hat_clf_sk = MLP_clf.predict_proba(x_train)

log_loss(y_train0, y_train_hat_clf_sk)

0.006959798980621651

In [8]:
print(y_train0[0])
print(y_train0[1])

5
0


In [9]:
print(y_train_hat_clf_sk[0])
print(y_train_hat_clf_sk[1])

[1.30383052e-13 3.21504558e-12 3.54627091e-10 2.87004393e-03
 1.71687203e-20 9.97129956e-01 4.86017969e-16 2.73805068e-11
 1.83986776e-12 1.42012103e-10]
[9.99998630e-01 1.59786641e-11 1.33555776e-06 2.81816908e-12
 2.88471499e-19 1.52875228e-12 1.17987721e-09 1.71159909e-12
 1.16440658e-13 3.37065415e-08]


In [10]:
print("Number of Layers: ", len(MLP_clf.coefs_))
print("Weights layer 1: ", MLP_clf.coefs_[0].size+MLP_clf.intercepts_[0].size)
print("Weights layer 2: ", MLP_clf.coefs_[1].size+MLP_clf.intercepts_[1].size)
print("Weights layer 3: ", MLP_clf.coefs_[2].size+MLP_clf.intercepts_[2].size)


Number of Layers:  3
Weights layer 1:  78500
Weights layer 2:  10100
Weights layer 3:  1010


In [11]:
print(MLP_clf.out_activation_)
print(MLP_clf._label_binarizer.y_type_)


softmax
multiclass
